In [9]:
import sys
import pandas as pd
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 
 
class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수
        rqField = [0, 4, 20]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        print("통신상태", rqStatus, self.objRq.GetDibMsg1())
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
#            print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
        print('전 종목 코드 %d, 거래소 %d, 코스닥 %d' % (len(allcodelist), len(codeList), len(codeList2)))
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
        code_list=[]
        name_list=[]
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
 
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))
        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print('%s 상장주식수: %s, 시가총액 %s' %(name, format(listed, ','), format(markettot, ',')))
            code_list.append(item[0])
            name_list.append(name)
        dict1={'code' : code_list, 'name' : name_list}
        df=pd.DataFrame(dict1,columns=['code','name'])
        df.to_csv('siga.csv', index=False)
if __name__ == "__main__":
    objMarketTotal = CMarketTotal()
    objMarketTotal.GetAllMarketTotal()
    objMarketTotal.PrintMarketTotal()

전 종목 코드 3348, 거래소 1790, 코스닥 1558
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
전종목 시가총액 순 조회 (3348 종목)
삼성전자 상장주식수: 5,969,782, 시가총액 402,363,306,800,000
LG에너지솔루션 상장주식수: 234,000,000, 시가총액 97,578,000,000,000
SK하이닉스 상장주식수: 728,002,365, 시가총액 81,900,266,062,500
삼성바이오로직스 상장주식수: 71,174,000, 시가총액 59,643,812,000,000
삼성전자우 상장주식수: 822,886,700, 시가총액 48,797,181,310,000
NAVER 상장주식수: 164,049,085, 시가총액 47,000,062,852,500
삼성SDI 상장주식